# papeles package - institutions network analysis example

In this notebook, all institutions from Neurips papers are extracted and then an institutions network is created using the co-location of such institutions in the research papers. Data was obtained using the [neurips_crawler](https://github.com/glhuilli/neurips_crawler).

The `papeles` package is used to extract a clean version of the institutions by processing the front page of the research papers, identifying from there which institutions were involved in the research. Results shows that...



In [23]:
import os
import json
from collections import defaultdict

from tqdm.notebook import tqdm


from papeles.paper.neurips import get_key


## Loading the data

In [25]:
# These are files with encoding issues that were not parse correctly by the pdf_parser 
SKIP_FILES = [
    '5049-nonparametric-multi-group-membership-model-for-dynamic-networks.pdf_headers.txt',
    '4984-cluster-trees-on-manifolds.pdf_headers.txt',
    '5820-alternating-minimization-for-regression-problems-with-vector-valued-outputs.pdf_headers.txt',
    '9065-visualizing-and-measuring-the-geometry-of-bert.pdf_headers.txt'
    '4130-implicit-encoding-of-prior-probabilities-in-optimal-neural-populations.pdf_headers.txt',
    '7118-local-aggregative-games.pdf_headers.txt'
]

NEURIPS_ANALYSIS_DATA_PATH = '/var/data/neurips_analysis'

file_lines = defaultdict(list)
for filename in tqdm(os.listdir(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_headers/')), 'loading files'):
    if filename in SKIP_FILES:
        continue
    with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, './files_headers/', filename), 'r') as f:
        for line in f.readlines():
            file_lines[get_key(filename)].append(line.strip())
            
metadata_path = os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_metadata/')

metadata = {}
for filename in tqdm(os.listdir(metadata_path), 'loading metadata'):
    with open(os.path.join(metadata_path, filename), 'r') as f: # open in readonly mode
        for line in f.readlines():
            data = json.loads(line)
            metadata[get_key(data['pdf_name'])] = data


In [29]:
from papeles.paper.neurips import institutions

from collections import Counter 
import itertools

inst_counter = Counter()
for file, lines in list(file_lines.items()):
    file_institutions = []
    for institution in institutions.parse_institutions(lines):
        if institution:
            name = institutions.fix_typo(institution[-1])
            names = institutions.fix_institution_parsing(name)
            for name in names:
                file_institutions.append(institutions.fix_institution_name(name))
    unique_file_institutions = list(set(file_institutions))
    inst_counter.update(unique_file_institutions)


cleaned = len(sorted([x for x in inst_counter.items() if x[1] > 0 and x[0]], key= lambda x: x[1], reverse=True))
total = sum([x[1] for x in inst_counter.items() if x[1] > 0 and x[0]])

print(f'current institutions: {cleaned}')
print(f'total raw institutions: {total}')
print(f'clean-up fraction: {"{:0.2f}".format(1 - cleaned / total)}')

current institutions: 2663
total raw institutions: 8072
clean-up fraction: 0.67


In [33]:
from collections import Counter 
import itertools

import networkx as nx


def graph_to_output(graph):
    output_graph = {'nodes': [], 'links': []}    
    nodes = set()
    for e in graph.edges():
        output_graph['links'].append({'source': e[0], 'target': e[1], 'value': graph[e[0]][e[1]]['weight']})
        nodes.add(e[0])
        nodes.add(e[1])
    output_graph['nodes'] = [{'id': n, 'group': 1} for n in list(nodes)]
    with open('neurips_institutions.json', 'w') as f:
        json.dump(output_graph, f)        

graph = nx.Graph()

filtered_institutions = set([x[0] for x in inst_counter.items() if x[1] > 1 and x[0]])


for file, lines in list(file_lines.items()):
    file_institutions = []
    for institution in parse_institutions(lines):
        if institution:
            name = institutions.fix_typo(institution[-1])
            names = institutions.fix_institution_parsing(name)
            for name in names:
                if institutions.fix_institution_name(name) in filtered_institutions:
                    file_institutions.append(institutions.fix_institution_name(name))
    unique_file_institutions = list(set(file_institutions))
    
    for i, j in itertools.combinations(unique_file_institutions, 2):
        if graph.has_edge(i, j):
            graph[i][j]['weight'] += 1
        else:
            graph.add_edge(i, j, weight=1)
            graph.add_edge(j, i, weight=1)

print(nx.info(graph))
graph_to_output(graph)


Name: 
Type: Graph
Number of nodes: 365
Number of edges: 1537
Average degree:   8.4219


In [37]:
import matplotlib.pyplot as plt
import community

def plot_graph(graph):
    node_size = []
    for node, degree in graph.degree():
        if degree < 5:
            node_size.append(10)
        elif 5 <= degree < 10:
            node_size.append(40)
        else: 
            node_size.append(90)
    
    partition = community.best_partition(graph)
    
    edge_colors = []
    edge_width = []
    for i, j in graph.edges():
        if graph[i][j]['weight'] < 5:
            edge_colors.append('gray')
            edge_width.append(0.1)
        elif 5 <= graph[i][j]['weight'] < 10:
                edge_colors.append('black')                
                edge_width.append(1)
        elif 5 <= graph[i][j]['weight'] < 15:
                edge_colors.append('blue')                
                edge_width.append(1)
        else:
            edge_colors.append('red')
            edge_width.append(6)
            
    plt.figure(1, figsize=(120,120)) 
    nx.draw(graph, 
            with_labels=True, 
            node_color=list(partition.values()),  #node_color_map, 
            node_size=node_size, 
            font_size=8, 
            edge_color=edge_colors, 
            width=edge_width, 
            alpha=0.8)
    plt.savefig("institutions_graph.png")

    
plot_graph(graph)

In [38]:
centrality = nx.eigenvector_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('microsoft research', 0.2432),
 ('carnegie mellon university', 0.2383),
 ('university of california berkeley', 0.2298),
 ('mit', 0.2287),
 ('princeton university', 0.1906),
 ('stanford university', 0.1891),
 ('google', 0.1862),
 ('google research', 0.175),
 ('harvard university', 0.1622),
 ('university of pennsylvania', 0.1609),
 ('columbia university', 0.1578),
 ('university of texas at austin', 0.1532),
 ('google brain', 0.1446),
 ('university of washington', 0.1427),
 ('ibm research', 0.1407),
 ('university of oxford', 0.1391),
 ('new york university', 0.1383),
 ('university of california san diego', 0.137),
 ('university college london', 0.1298),
 ('university of michigan', 0.1271),
 ('deepmind', 0.1245),
 ('duke university', 0.1235),
 ('tsinghua university', 0.12),
 ('university of cambridge', 0.1193),
 ('university of california los angeles', 0.1158),
 ('toyota technological institute at chicago', 0.1144),
 ('university of illinois at urbana-champaign', 0.114),
 ('cornell unive

In [41]:
centrality = nx.katz_centrality_numpy(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('inria', 0.2136),
 ('ecole normale superieure', 0.1634),
 ('university of oxford', 0.1406),
 ('eth zürich', 0.1383),
 ('university of cambridge', 0.135),
 ('cnrs', 0.131),
 ('max planck institute for intelligent systems', 0.1309),
 ('deepmind', 0.1248),
 ('facebook ai research', 0.1083),
 ('university of copenhagen', 0.1056),
 ('alan turing institute', 0.1052),
 ('university college london', 0.101),
 ('sorbonne universite', 0.0932),
 ('max planck institute', 0.0928),
 ('ecole polytechnique', 0.0883),
 ('universitat pompeu fabra', 0.0871),
 ('imperial college london', 0.0855),
 ('new york university', 0.0807),
 ('university of edinburgh', 0.0771),
 ('epfl', 0.0768),
 ('google brain', 0.0767),
 ('university of sydney', 0.0765),
 ('mit', 0.0733),
 ('ist austria', 0.0706),
 ('uber', 0.0704),
 ('university of genoa', 0.0696),
 ('leiden university', 0.0674),
 ('australian national university', 0.0654),
 ('jst presto', 0.0653),
 ('university of alberta', 0.0649),
 ('university of warwick', 

In [42]:
centrality = nx.closeness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('microsoft research', 0.5165),
 ('carnegie mellon university', 0.5107),
 ('mit', 0.5107),
 ('university of california berkeley', 0.5071),
 ('stanford university', 0.4911),
 ('princeton university', 0.4743),
 ('google research', 0.4706),
 ('google', 0.4663),
 ('university of oxford', 0.4645),
 ('harvard university', 0.461),
 ('university college london', 0.4604),
 ('columbia university', 0.4574),
 ('new york university', 0.4551),
 ('university of pennsylvania', 0.45),
 ('ibm research', 0.45),
 ('university of cambridge', 0.4489),
 ('deepmind', 0.4483),
 ('google brain', 0.4483),
 ('university of california san diego', 0.4478),
 ('university of washington', 0.4467),
 ('university of texas at austin', 0.4461),
 ('duke university', 0.4434),
 ('inria', 0.4428),
 ('cornell university', 0.439),
 ('university of california los angeles', 0.439),
 ('eth zürich', 0.4364),
 ('university of illinois at urbana-champaign', 0.4348),
 ('tsinghua university', 0.4348),
 ('university of michigan', 0.433

In [43]:
centrality = nx.information_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)

NetworkXError: Graph not connected.

In [44]:
centrality = nx.betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('mit', 0.1453),
 ('microsoft research', 0.1286),
 ('university of california berkeley', 0.1044),
 ('carnegie mellon university', 0.1032),
 ('stanford university', 0.0765),
 ('university college london', 0.0638),
 ('university of oxford', 0.0513),
 ('google', 0.0511),
 ('inria', 0.0439),
 ('columbia university', 0.0382),
 ('university of texas at austin', 0.0363),
 ('princeton university', 0.0354),
 ('university of cambridge', 0.0351),
 ('google research', 0.0346),
 ('cnrs', 0.0336),
 ('university of california los angeles', 0.0333),
 ('new york university', 0.0318),
 ('cornell university', 0.0307),
 ('harvard university', 0.0291),
 ('duke university', 0.0274),
 ('deepmind', 0.026),
 ('ntt communication science laboratories', 0.0257),
 ('ibm research', 0.0246),
 ('university of pennsylvania', 0.0232),
 ('university of illinois at urbana-champaign', 0.0228),
 ('university of alberta', 0.0226),
 ('eth zürich', 0.0225),
 ('rutgers university', 0.0216),
 ('chinese university of hong kong'

In [45]:
centrality = nx.current_flow_betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


NetworkXError: Graph not connected.

In [46]:
centrality = nx.communicability_betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


/Users/gastonlhuillier/Personal/papeles/.python/papeles/lib/python3.8/site-packages/networkx/algorithms/centrality/subgraph_alg.py:246: RuntimeWarning: invalid value encountered in true_divide
  B = (expA - scipy.linalg.expm(A.A)) / expA


[('australian national university', nan),
 ('nicta', nan),
 ('university of alberta', nan),
 ('cifar', nan),
 ('university of amsterdam', nan),
 ('microsoft research', nan),
 ('university of massachusetts amherst', nan),
 ('ecole centrale paris', nan),
 ('inria', nan),
 ('cornell university', nan),
 ('university of california berkeley', nan),
 ('princeton university', nan),
 ('ebay inc', nan),
 ('stanford university', nan),
 ('allen institute for ai', nan),
 ('university of texas at austin', nan),
 ('university of illinois at urbana-champaign', nan),
 ('openai', nan),
 ('brown university', nan),
 ('bar-ilan university', nan),
 ('harvard university', nan),
 ('university of verona', nan),
 ('indian institute of science', nan),
 ('chalmers university of technology', nan),
 ('toyota technological institute at chicago', nan),
 ('max planck institute for intelligent systems', nan),
 ('alan turing institute', nan),
 ('sun yat-sen university', nan),
 ('chinese university of hong kong', nan),
 

In [47]:
hubs, authorities = nx.hits(graph)

In [48]:
sorted([(v, float('{:0.4f}'.format(c))) for v, c in hubs.items()], key=lambda x: x[1], reverse=True)

[('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
 ('duke university', 0.0119),
 ('deepmind', 0.0119),
 ('ibm research', 0.0112),
 ('university college london', 0.0107),
 ('georgia institute of technology', 0.0105),
 ('tsinghua university', 0.0102),
 ('university of michigan', 0.0101),
 ('toyota technological institute at chicago', 0.0099),
 ('t

In [49]:
sorted([(v, float('{:0.4f}'.format(c))) for v, c in authorities.items()], key=lambda x: x[1], reverse=True)

[('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
 ('duke university', 0.0119),
 ('deepmind', 0.0119),
 ('ibm research', 0.0112),
 ('university college london', 0.0107),
 ('georgia institute of technology', 0.0105),
 ('tsinghua university', 0.0102),
 ('university of michigan', 0.0101),
 ('toyota technological institute at chicago', 0.0099),
 ('t